In [1]:
import os 


In [2]:
%pwd

'c:\\Users\\Asus F15\\Text-summarization\\research'

In [6]:
os.chdir("../")

In [8]:
%pwd

'c:\\Users\\Asus F15\\Text-summarization'

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metrics_file_name: Path
   


In [16]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml, create_directories

In [17]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=Path(config.root_dir),
            data_path=Path(config.data_path),
            model_path=Path(config.model_path),
            tokenizer_path=Path(config.tokenizer_path),
            metrics_file_name=Path(config.metrics_file_name)

        ) 
        return model_evaluation_config


In [18]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset,load_from_disk,load_metric
import torch
import pandas as pd
from tqdm import tqdm

d:\project text summarization\envs\texts\lib\site-packages\transformers\utils\hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


ImportError: cannot import name 'load_metric' from 'datasets' (d:\project text summarization\envs\texts\lib\site-packages\datasets\__init__.py)

In [19]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def gemerate_batch_sized_chunks(self,list_of_elements,batch_size=32):
        """        Splits a list into smaller chunks of specified batch size.
        """       
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i:i + batch_size]

    def calculate_metrics(self,datasets,metric,model,tokenizer,batch_size=16,device="cuda" if torch.cuda.is_available() else "cpu",
                          column_text="articles",column_summary="hghlights"):
        article_batches=list(self.gemerate_batch_sized_chunks(datasets[column_text],batch_size))
        summary_batches=list(self.gemerate_batch_sized_chunks(datasets[column_summary],batch_size))

        for article_batch,target_batch in tqdm(zip(article_batches,summary_batches),total=len(article_batches)):

            inputs=tokenizer(article_batch,max_length=1024,truncation=True,padding="max_length",return_tensors="pt")

            summaries =model.generate(
                input_ids=inputs["input_ids"].to(device),
                attention_mask=inputs["attention_mask"].to(device),
                length_penalty=0.8,
                num_beams=4,
                max_length=128)
            
            ''' parameter for length_penalty ensures that the model does not generate very long summaries'''

            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summaries]

            metric.add_batch(predictions=decoded_summaries, references=target_batch)

            #finaly compute the return ROUGE score
            score = metric.compute(use_stemmer=True)
            return score
        
        def evaluate(self):
            device = "cuda" if torch.cuda.is_available() else "cpu"
            tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
            model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

            # load the dataset
            datasets = load_from_disk(self.config.data_path)


            rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

            roufge_metric = load_metric('rouge')

            score = self.calculate_metric_on_test_ds(dataset_samsum_pt['test'][0:10],roufge_metric,model,tokenizer,batch_size=2,column_text="dialogue",column_summary="summary")

            rouge_dict=dict((rn,score[rn].mid.fmeasure) for rn in rouge_names)

            df =pd.DataFrame(rouge_dict,index=['pegasus'])
            df.to_csv(self.config.metrics_file_name,index=False)

NameError: name 'torch' is not defined

In [ ]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate()
except Exception as e:
    raise e